<a href="https://colab.research.google.com/github/arthurrferroni/UNIFEI/blob/main/Blockchain%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulando uma Aplicação *Iot* com *Blockchain* 

Baseado no código [IoTblockchain de Sumaya Altamimi](https://github.com/suumaya/Blockchain-IoT/blob/master/IoTblockchain.ipynb)

## Implementação em Python

In [ ]:
!pip install pyCrypto

In [ ]:
import hashlib #codificação em sha256
import random
import string
import json
import binascii
import Crypto
import Crypto.Random #gera chaves públicas e privadas
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

O primeiro passo é construir um minerador, que validará as cadeias construídas para o *Blockchain*.

Portanto, começaremos um pouco para trás e começaremos com a implementação do minerador. Para o exemplo aqui, usaremos a função hash SHA256 porque ela é prontamente implementada em python. Observe que o bitcoin usa [duas rodadas de SHA256](https://en.bitcoin.it/wiki/Hashcash) em vez de uma.

Portanto, nossa função hash transformará uma string de comprimento arbitrário em uma string de comprimento fixo de 64 caracteres hexadecimais. 

In [ ]:
def sha256(mensagem):
  return hashlib.sha256(mensagem.encode('ascii')).hexdigest()

def hash_iot(mensagem):
  return sha256(mensagem)

def minerador(mensagem, dificuldade=1):
  '''
  Retorna o HASH da soma da mensagem mais o nonce (número aleatório usado 1 vez).
  A dificuldade define o tamanho do nonce.
  Retorna:
    nonce: o número encontrado
    niter: número de iterações para encontra o nonce
  '''
  i = 0
  prefix = '1' * dificuldade
  while True:
    nonce = str(i)
    digest = hash_iot(mensagem + nonce)
    if digest.startswith(prefix):
      return nonce, i
    i += 1

Quanto mais você aumenta o número de líderes de que necessita, mais difícil se torna (em média) encontrar um nonce. No bitcoin, isso é chamado de dificuldade de mineração. Observe que o bitcoin não requer um número de dígitos iniciais,  e sim requer que o hash esteja abaixo de um determinado valor, mas é a mesma ideia.

Portanto, vamos definir duas funções que reutilizaremos mais tarde: uma para fazer um hash de uma string e outra para extrair um nonce de uma determinada string.

Agora, o processo de mineração é: dado um texto `x` arbitrário, encontre um nonce (número arbitrário único) `hash(x + nonce)` que produza um hash começando com um número de caracteres principais.

Por exemplo, aqui, iremos "minerar" um nonce de forma que o hash de nossa mensagem `'CEB-IoT'` quando concatenado com nosso nonce tenha pelo menos 2 principais.

In [ ]:
message = 'CEB-IoT'
for nonce in range(1000):
    digest = sha256(message + str(nonce))
    if digest.startswith('12'):
        print('Nonce encontrado = %d' % nonce)
        break
print(sha256(message + str(nonce)))

Variando a dificuldade de minerar o nonce de 1 para 3 a seguir.

In [ ]:
nonce, niter = minerador('CEB-IoT', dificuldade=1)
print('Nonce %s encontrado com %d iterações.' % (nonce,niter))

nonce, niter = minerador('CEB-IoT', dificuldade=3)
print('Nonce %s encontrado com %d iterações.' % (nonce,niter))

Como você pode ver neste exemplo, o número de iterações necessárias para uma dificuldade de 3 é muito maior do que para uma dificuldade de 1. 

Observe que ter sorte e obter uma string onde o primeiro nonce (0 neste caso) seria produzir a solução. 

Portanto, a dificuldade controla o número *médio* de tentativas. Gerenciando o risco de quebra do HASH.

## Emulação do Dispositivo

Cada dispositivo IoT deve ter um par de chaves: privada + pública.

A chave pública é usada para receber dados e a chave privada é usada para processar dados.

Ao assinar uma mensagem com nossa chave privada, qualquer outra pessoa pode verificar a assinatura usando nossa chave pública.

Observe que o blockchain real é mais complicado.

Um blockchain é um conjunto de vários pares de chaves privadas / públicas e um endereço não é diretamente a chave pública.

Isso garante melhor privacidade e segurança, mas para a proposta de implementação de IoT, usaremos uma única chave e usaremos a chave pública como o endereço.

In [ ]:
class Dispositivo():
  '''
  Dispositivo com par de chaves: pública e privada
  '''
  def __init__(self):
    random_gen = Crypto.Random.new().read
    self._private_key = RSA.generate(1024, random_gen)
    self._public_key = self._private_key.publickey()
    self._signer = PKCS1_v1_5.new(self._private_key)
        
  @property
  def endereco(self):
    '''Atalho para acessar a chave pública'''
    return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
    
  def assinatura(self, message):
    '''Assinatura da mensagem com a carteira'''
    h = SHA.new(message.encode('utf8'))
    return binascii.hexlify(self._signer.sign(h)).decode('ascii')
    

In [ ]:
def verifica_assinatura(disp_endereco, mensagem, assinatura):
  '''
  Verifica a assinatura correspondente a mensagem assinada pelo 
  dispositvo em seu endereço
  '''
  publica = RSA.importKey(binascii.unhexlify(disp_endereco))
  verifica = PKCS1_v1_5.new(publica)
  h = SHA.new(mensagem.encode('utf8'))
  return verifica.verify(h, binascii.unhexlify(assinatura))

### Teste do Dispositivo

Verifica se a assinatura da carteira funciona para o dispositivo.

In [ ]:
d1 = Dispositivo()
msg = 'CEB IoT 5G'
assi = d1.assinatura(msg)
print('assinatura =', assi)
print('\'%s\' [%s]' % (msg, verifica_assinatura(d1.endereco, msg, assi)))
msg = 'CEB IoT 4G'
print('\'%s\' [%s]' % (msg, verifica_assinatura(d1.endereco, msg, assi)))

## Processando o Envio de Mensagens

Usa-se um processo para trocar mensagens entre dispositivos, composto por:
1. Um **remetente**, que escreverá e assinará a mensagem.
2. Um **número de entradas**, que são outras saídas de mensagens. 
  - O destinatário de todos eles deve ser o dispositivo do remetente. 
  - Caso contrário, você pode ver os dados de outras pessoas.
3. Um **número de saídas**, cada uma especificando uma **mensagem** e um **destinatário**.

A classe *ProcessoEntrada* indica uma entrada por processo, apontando para a saída de outro processo.

In [ ]:
class ProcessoEntrada():
  def __init__(self, processo, indice_saida):
    self.processo = processo
    self.indice_saida = indice_saida
    assert 0 <= self.indice_saida < len(processo.saidas)
        
  def to_dict(self):
    d = {
        'processo': self.processo.hash(),
        'indice_saida': self.indice_saida
      }
    return d
    
  @property
  def saida_pai(self):
    return self.processo.saidas[self.indice_saida]

A classe *ProcessoSaida* indica saída para um processo, isso especifica a mensagem e um dispositivo de destinatário.

In [ ]:
class ProcessoSaida():
  def __init__(self, endereco_destino, mensagem):
    self.destino = endereco_destino
    self.mensagem = mensagem
        
  def to_dict(self):
    d = {
        'endereco_destino': self.destino,
        'mensagem': self.mensagem
      }
    return d

A classe *Processo* cria o processo de envio de mensagens do dispositivo.

In [ ]:
class Processo():
  def __init__(self, dispositivo, entradas, saidas):
    self.entradas = entradas
    self.saidas = saidas
    self.assinatura = dispositivo.assinatura(
        json.dumps(
            self.to_dict(inclue_assinatura=False)))
        
  def to_dict(self, inclue_assinatura=True):
    d = {
      "entradas": list(map(ProcessoEntrada.to_dict, self.entradas)),
      "saidas": list(map(ProcessoSaida.to_dict, self.saidas))
    }
    if inclue_assinatura:
      d["assinatura"] = self.assinatura
    return d
    
  def hash(self):
    return hash_iot(json.dumps(self.to_dict()))

Assim todos os processos precisam de um pai, desta forma, é necessário definir uma raiz de origem na hierarquia. Essa ponto de partida será o *ProcessoRaiz*.

In [ ]:
class ProcessoRaiz(Processo):
  def __init__(self, endereco_destino, mensagem='primeiro processo'):
    self.entradas = []
    self.saidas = [ProcessoSaida(endereco_destino, mensagem)]
    self.assinatura = 'origem'
        
  def to_dict(self, inclue_assinatura=False):
    assert not inclue_assinatura, 'Não pode incluir assinatura no processo raiz'
    return super().to_dict(inclue_assinatura=False)

### Testando os Processos Entre 3 Sensores

In [ ]:
Corrente = Dispositivo()
Tensao = Dispositivo()
Ilumina = Dispositivo()

p1 = ProcessoRaiz(Corrente.endereco,'2.1678A')

p2 = Processo(Corrente, [ProcessoEntrada(p1, 0)], [ProcessoSaida(Tensao.endereco, '121.71Vac')])

p3 = Processo(Corrente, [ProcessoEntrada(p1, 0)], [ProcessoSaida(Tensao.endereco, '119.25Vac'), ProcessoSaida(Ilumina.endereco, '1024Lu')])

p4 = Processo(Ilumina, [ProcessoEntrada(p3, 1)], [ProcessoSaida(Tensao.endereco, '128.12Vac')])

p5 = Processo(Tensao,[ProcessoEntrada(p2, 0), ProcessoEntrada(p3, 0),ProcessoEntrada(p4, 0)],[ProcessoSaida(Ilumina.endereco, '971Lu')])

processos = [p1, p2, p3, p4 , p5]
processos

No blockchain, você nunca armazena todas as mensagens em seu dispositivo. 

Em vez disso, você percorre toda a cadeia de processos para visualizar todas as suas mensagens. 

A função *ver_mensagens* list as mensagens nos processos de um endereço fornecido.

In [ ]:
def ver_mensagens(endereco_dispositivo, processos):
  mensagens = []
  for p in processos:
    for ps in p.saidas:
        if ps.destino == endereco_dispositivo:
            mensagens.append(ps.mensagem)
  return mensagens

### Listando Mensagens dos Processos

In [ ]:
print("Mensagens da Corrente", ver_mensagens(Corrente.endereco, processos),'\n')
print("Mensagens da Tensão" , ver_mensagens(Tensao.endereco, processos),'\n')
print("Mensagens da Iluminação" , ver_mensagens(Ilumina.endereco, processos),'\n')

### Verificação de Processo Válido

Todas as mensagens de um endereço de dispositivo são enviadas por esse endereço de dispositivo. Deve verificar então:
- Todas as entradas fazem parte do proprietário do dispositivo;
- O processo deve ser assinado pelo proprietário do dispositivo.

In [ ]:
def verifica_processo(Processo):

  pr_mensagem = json.dumps(Processo.to_dict(inclue_assinatura=False)) 

  # processo raiz não é validado aqui
  if isinstance(Processo, ProcessoRaiz):
    return True

  # 1 - Verifica os processos interligados com recursividade
  for px in Processo.entradas:
    if not verifica_processo(px.processo):
        print('Processo pai inválido')
        return False
    
  endereco_entrada_inicial = Processo.entradas[0].saida_pai.destino    
  for prin in Processo.entradas[1:]:
    if prin.saida_pai.destino != endereco_entrada_inicial:
      print('Processo de entrada pertence a multiplos dispositivos:',
            prin.saida_pai.destino, 
            ';',
            endereco_entrada_inicial)
    return False
    
  # 2 - Verifica assinatura
  if not verifica_assinatura(endereco_entrada_inicial, pr_mensagem, Processo.assinatura):
    print('Assinatura do processo inválida, endereço não está incorreto?')
    return False

  return True

## Fazendo o Processo p3 ser inválido para Teste

Criando uma corrente de mensagens incorreta no processo 3.

Onde o dispositivo Tensão sobrepoem o dispositivo Corrente.

In [ ]:
p1 = ProcessoRaiz(Corrente.endereco,'2.2541A')

p2 = Processo(Corrente, [ProcessoEntrada(p1, 0)], [ProcessoSaida(Ilumina.endereco, '1003Lu')])

p3 = Processo(Tensao, [ProcessoEntrada(p1, 0)],[ProcessoSaida(Ilumina.endereco, '1102Lu')])

processos = [p1, p2, p3]
processos

### Validando os Processos

In [ ]:
print('Verifica p1:')
print('   ',verifica_processo(p1))
print('Verifica p2:')
print('   ',verifica_processo(p2))
print('Verifica p3:')
print('   ',verifica_processo(p3))

## Inserindo os Processos em Blocos

Após os seguintes passos anteriores:
- Definir um dispositivo com chaves privadas e públicas;
- Criar processos entre os dispositivos;
- Verificar se os processos são válidos por assinatura.

Passo atual é inserir os processos em blocos e realizar a mineração de exploração.

A mineração do bloco consiste:
- Verificar os processos no bloco;
- Encontrar o Nonce conforme os números zero na frente do hash do bloco.

In [ ]:
# dificuldade da mineração
dificuldade = 2 

class Bloco():
  def __init__(self, processos_bloco_atual, bloco_anterior, endereco_minerador, verificar=True):
    self.processos_bloco_atual = [ProcessoRaiz(endereco_minerador)] + processos_bloco_atual
    self.bloco_anterior = bloco_anterior

    if verificar:
      # gera erro de verificação
      assert all(map(verifica_processo, processos_bloco_atual))

    json_block = json.dumps(self.to_dict(inclue_hash=False))
    self.nonce, _ = minerador(json_block, dificuldade)
    self.hash = hash_iot(json_block + self.nonce)
  
  def to_dict(self, inclue_hash=True):
    d = {
      "processos": list(map(Processo.to_dict, self.processos_bloco_atual)),
      "bloco_anterior": self.bloco_anterior.hash,
      }
    if inclue_hash:
        d["nonce"] = self.nonce
        d["hash"] = self.hash
    return d

# Bloco sem anterior
class BlocoInicial(Bloco):
  def __init__(self, endereco_minerador):
    super(BlocoInicial, self).__init__(
        processos_bloco_atual=[], 
        bloco_anterior=None, 
        endereco_minerador=endereco_minerador,
        )
  def to_dict(self, inclue_hash=True):
    d = {
      "processos": [],
      "bloco_anterior": True,
      }
    if inclue_hash:
        d["nonce"] = self.nonce
        d["hash"] = self.hash
    return d

### Verificação de Blocos

Passos de verificação de blocos:
 1. Verificar o hash **inicia com** um número necessário de **1** (2 neste caso);
 2. Verificar **todos os processos** são **válidos**;
 3. Verificar o **primeiro processo** do bloco é um **ProcessoRaiz**.

In [ ]:
def verifica_bloco(bloco, primeiro_bloco, saidas_usadas=None):
  '''
  Argumentos:
     bloco = bloco a ser validado
     primeiro_bloco = primeiro bloco a ser compartilhado com todos
     saidas_usadas = lista de saídas usadas nos processos dos bloco sobre este
  '''
  if saidas_usadas is None:
    saidas_usadas = set()
  
  # 1 - Verifica hash
  prefixo = '1' * dificuldade
  if not bloco.hash.startswith(prefixo):
    print('O Hash do Bloco',bloco.hash,'não corresponde com o prefixo',prefixo)
    return False
  
  # 2.a - Processos são válidos
  if not all(map(verifica_processo, bloco.processos_bloco_atual)):
    return False
  
  # 2.b - Verificando blocos após primeiro bloco
  if not (bloco.hash == primeiro_bloco.hash):
    if not verifica_bloco(bloco.bloco_anterior, primeiro_bloco, saidas_usadas):
      print('Falha na autenticação do bloco posterior!')
      return False
  
  # 3.a - Verificar processo inicial é Raiz
  pr0 = bloco.processos_bloco_atual[0]
  if not isinstance(pr0, ProcessoRaiz):
    print('Processo inicial não é Raiz!')
    return False

  # 3.b - Verificar os demais processos
  for i, pr in enumerate(bloco.processos_bloco_atual):
    if i == 0:
      if not isinstance(pr, ProcessoRaiz):
        print('Processo inicial não é Raiz!')
        return False  
    elif isinstance(pr, ProcessoRaiz):
      print('Processo Raiz está na posição',i,'!')
      return False
  
  return True

## Validando os Blocos em Teste

In [ ]:
Corrente = Dispositivo()
Tensao = Dispositivo()
Ilumina = Dispositivo()

# corrente minerada no primeiro bloco
primeiro_bloco = BlocoInicial(endereco_minerador=Corrente.endereco)

# processo raiz
p1 = primeiro_bloco.processos_bloco_atual[0]

p2 = Processo(
    Corrente,
    [ProcessoEntrada(p1, 0)],
    [ProcessoSaida(Tensao.endereco, '126.9V'), 
     ProcessoSaida(Ilumina.endereco, '923Lu')],
     )

p3 = Processo(
    Ilumina,
    [ProcessoEntrada(p2, 1)], 
    [ProcessoSaida(Tensao.endereco, '128.4V')],
    )

p4 = Processo(
    Tensao, 
    [ProcessoEntrada(p2, 0)],
    [ProcessoSaida(Ilumina.endereco, '1234Lu')]
)

# Tensão minerada no bloco 1
bloco1 = Bloco([p2,p3], bloco_anterior=primeiro_bloco, endereco_minerador=Tensao.endereco)

# Iluminação minerada no bloco 2
bloco2 = Bloco([p4], bloco_anterior=bloco1, endereco_minerador=Ilumina.endereco) 

print("Hash 1o bloco:", primeiro_bloco.hash)
print("Hash bloco 1 :", bloco1.hash)
print("Hash bloco 2 :", bloco2.hash)
print('====')
print('Verificar 1o bloco')
print('   ',verifica_bloco(primeiro_bloco, primeiro_bloco))
print('Verificar bloco 1')
print('   ',verifica_bloco(bloco1, primeiro_bloco))
print('Verificar bloco 2')
print('   ',verifica_bloco(bloco2, primeiro_bloco))

## Gerando o *Blockchain*

In [ ]:
def processa_cadeia(bloco, primeiro_bloco):
  '''
  Coleta recursivamente os processos do bloco e 
  seus blocos anteriores para simular uma mensagem em Blockchain
  '''
  processos = [] + bloco.processos_bloco_atual
  if bloco.hash != primeiro_bloco.hash:
    processos += processa_cadeia(bloco.bloco_anterior, primeiro_bloco)

  return processos

### Processando Blockchain

In [ ]:
processos = processa_cadeia(bloco2, primeiro_bloco)

print('Mensagens de Corrente:', ver_mensagens(Corrente.endereco, processos))
print('Mensagens de Tensão:', ver_mensagens(Tensao.endereco, processos))
print('Mensagens de Iluminação:', ver_mensagens(Ilumina.endereco, processos))